In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [5]:
import os
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate the user in the Colab environment
auth.authenticate_user()

# Configure the Google Docs API service
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']
creds, _ = default()

docs_service = build('docs', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

print("✅ Authentication and configuration complete.")

✅ Authentication and configuration complete.


In [11]:
import re
from googleapiclient.errors import HttpError

# Store the meeting notes markdown in a variable
markdown_notes = """
# Product Team Sync - May 15, 2023
## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)
## Agenda
### 1. Sprint Review
* Completed Features
* User authentication flow
* Dashboard redesign
* Performance optimization
* Reduced load time by 40%
* Implemented caching solution
* Pending Items
* Mobile responsive fixes
* Beta testing feedback integration
### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
* Navigation confusion
* Color contrast issues
### 3. Next Sprint Planning
* Priority Features
* Payment gateway integration
* User profile enhancement
* Analytics dashboard
* Technical Debt
* Code refactoring
* Documentation updates
## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal
## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders
## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

def convert_markdown_to_google_doc(notes):
    """
    Convert Markdown meeting notes into a styled Google Doc.
    Markdown markers (#, ##, ###, - , * , - [ ] ) are removed from the text
    before insertion so they are not visible in the final document.
    Styles (headings, bullets, checkboxes, mentions, footer) are then applied.
    """
    try:
        # Create the document
        first_line = notes.strip().split('\n')[0].strip()
        title = first_line.replace('# ', '').strip()
        doc = docs_service.documents().create(body={'title': f"Meeting Notes: {title}"}).execute()
        doc_id = doc['documentId']
        print(f"📄 Document created. ID: {doc_id}")

        # Preprocess: build cleaned lines and metadata for styling
        raw_lines = notes.strip().split('\n')
        cleaned_lines = []
        meta = []  # per-line metadata for styling and ranges

        for line in raw_lines:
            original = line
            working = line
            info = {
                'heading': None,          # 1/2/3 or None
                'is_hr': False,           # horizontal rule
                'is_footer': False,       # footer lines
                'is_bullet': False,       # plain bullet
                'is_checkbox': False,     # checkbox bullet
                'indent_spaces': 0,       # leading spaces count
                'mentions': []            # list of (start, end) in cleaned text
            }

            # Compute leading spaces (indent for nested lists)
            indent_spaces = len(working) - len(working.lstrip(' '))
            info['indent_spaces'] = indent_spaces

            # Heading detection (strip markers)
            if working.startswith('### '):
                info['heading'] = 3
                working = working[4:]
            elif working.startswith('## '):
                info['heading'] = 2
                working = working[3:]
            elif working.startswith('# '):
                info['heading'] = 1
                working = working[2:]

            # Horizontal rule
            if working.strip().startswith('---'):
                info['is_hr'] = True
                # Keep a visible paragraph (empty) to apply border
                working = ''

            # Footer detection
            if working.startswith('Meeting recorded by:') or working.startswith('Duration:'):
                info['is_footer'] = True

            # Bullet / checkbox detection (strip markers from the visible text)
            stripped = original.lstrip(' ')
            if stripped.startswith('- [ ] '):
                info['is_checkbox'] = True
                # Remove the checkbox literal but keep the original indent
                working = ' ' * indent_spaces + stripped[len('- [ ] '):]
            elif stripped.startswith('- '):
                info['is_bullet'] = True
                working = ' ' * indent_spaces + stripped[2:]
            elif stripped.startswith('* '):
                info['is_bullet'] = True
                working = ' ' * indent_spaces + stripped[2:]

            # Mentions on the cleaned text
            for m in re.finditer(r'(@\w+)', working):
                info['mentions'].append((m.start(), m.end()))

            cleaned_lines.append(working)
            meta.append(info)

        # Insert the cleaned text
        full_text = '\n'.join(cleaned_lines) + '\n'
        docs_service.documents().batchUpdate(
            documentId=doc_id,
            body={'requests': [{'insertText': {'location': {'index': 1}, 'text': full_text}}]}
        ).execute()

        # Build style requests using absolute ranges on the cleaned text
        requests = []
        current_index = 1  # start of inserted content

        for i, line in enumerate(cleaned_lines):
            info = meta[i]
            line_with_nl = line + '\n'
            line_len = len(line_with_nl)

            # Headings
            if info['heading'] in (1, 2, 3):
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': current_index, 'endIndex': current_index + line_len},
                        'paragraphStyle': {'namedStyleType': f'HEADING_{info["heading"]}'},
                        'fields': 'namedStyleType'
                    }
                })

            # Bullets / Checkboxes
            if info['is_bullet'] or info['is_checkbox']:
                preset = 'BULLET_CHECKBOX' if info['is_checkbox'] else 'BULLET_DISC_CIRCLE_SQUARE'
                requests.append({
                    'createParagraphBullets': {
                        'range': {'startIndex': current_index, 'endIndex': current_index + line_len},
                        'bulletPreset': preset
                    }
                })

                # Indentation (approx.: every 2 spaces -> 18pt)
                if info['indent_spaces'] > 0:
                    requests.append({
                        'updateParagraphStyle': {
                            'range': {'startIndex': current_index, 'endIndex': current_index + line_len},
                            'paragraphStyle': {
                                'indentStart': {'magnitude': 18 * (info['indent_spaces'] / 2.0), 'unit': 'PT'}
                            },
                            'fields': 'indentStart'
                        }
                    })

            # Horizontal rule (bottom border on the paragraph)
            if info['is_hr']:
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': current_index, 'endIndex': current_index + line_len},
                        'paragraphStyle': {
                            'borderBottom': {
                                'width': {'magnitude': 1, 'unit': 'PT'},
                                'padding': {'magnitude': 3, 'unit': 'PT'},
                                'dashStyle': 'SOLID',
                                'color': {'color': {'rgbColor': {'red': 0.0, 'green': 0.0, 'blue': 0.0}}}
                            }
                        },
                        'fields': 'borderBottom'
                    }
                })

            # Footer distinct style
            if info['is_footer']:
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': current_index, 'endIndex': current_index + line_len},
                        'paragraphStyle': {'alignment': 'END', 'spaceAbove': {'magnitude': 6, 'unit': 'PT'}},
                        'fields': 'alignment,spaceAbove'
                    }
                })
                requests.append({
                    'updateTextStyle': {
                        'range': {'startIndex': current_index, 'endIndex': current_index + line_len},
                        'textStyle': {'italic': True,
                                      'foregroundColor': {'color': {'rgbColor': {'red': 0.4, 'green': 0.4, 'blue': 0.4}}}},
                        'fields': 'italic,foregroundColor'
                    }
                })

            # Mentions styling on cleaned text
            for (s, e) in info['mentions']:
                requests.append({
                    'updateTextStyle': {
                        'range': {'startIndex': current_index + s, 'endIndex': current_index + e},
                        'textStyle': {
                            'bold': True,
                            'foregroundColor': {'color': {'rgbColor': {'red': 0.15, 'green': 0.35, 'blue': 0.80}}}
                        },
                        'fields': 'bold,foregroundColor'
                    }
                })

            current_index += line_len  # advance to next paragraph

        if requests:
            docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()

        print("✅ Formatting applied successfully.")
        print(f"🔗 https://docs.google.com/document/d/{doc_id}")

    except HttpError as error:
        print(f"❌ An error occurred: {error}")

# Run the main function
convert_markdown_to_google_doc(markdown_notes)

📄 Document created. ID: 1gfTP1RQ_SkQGCG4-PEznwYNec90UlTLhEEU3gY5YjeY
✅ Formatting applied successfully (no visible markdown markers).
🔗 https://docs.google.com/document/d/1gfTP1RQ_SkQGCG4-PEznwYNec90UlTLhEEU3gY5YjeY
